### Find similar items by keyword and send notification by e-mail

In [3]:
import json
import requests
from bs4 import BeautifulSoup
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
key ='Your Key'

#### Define function to send results

In [26]:
def sendemail(url, title, price):
    msg = MIMEMultipart()
    msg['From'] = 'From E-mail'
    msg['To'] = 'To E-mail'
    password = 'Password'
    msg['Subject'] = 'Item found!'
    body = "<a href= \"" + url + "\">" + title + "</a><br><p>Price: " + price + "</p>"
    msg.attach(MIMEText(body, 'html'))

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(msg['From'], password)
    server.sendmail(msg['From'], msg['To'], msg.as_string())
    server.quit()

#### Search items by keywords and send results using fuction above

In [30]:
searches = []

with open('searches.txt', 'r') as searchfile:
    searches = searchfile.readlines()
with open('itemid.txt', 'r') as itemfile:
    itemids = itemfile.read().splitlines()
    

for item in searches:
    search = item.split(',')[0]
    max_price = item.split(',')[1]
    url = ('http://svcs.ebay.com/services/search/FindingService/v1\
?OPERATION-NAME=findItemsByKeywords\
&sortOrder=PricePlusShippingLowest\
&buyerPostalCode=11235&SERVICE-VERSION=1.13.0\
&SECURITY-APPNAME=' + key +
'&RESPONSE-DATA-FORMAT=JSON\
&REST-PAYLOAD\
&itemFilter(0).name=Condition\
&itemFilter(0).value=New\
&itemFilter(1).name=MaxPrice\
&itemFilter(1).value=' + max_price +\
'&itemFilter(1).paramName=Currency\
&itemFilter(1).paramValue=USD\
&keywords=' + search)
    url = url.replace(' ', '%20')
    apiResult = requests.get(url)
    parseddoc = apiResult.json()
    for item in (parseddoc["findItemsByKeywordsResponse"][0]["searchResult"][0]["item"]):
        itemid = item['itemId'][0]
        url = item['viewItemURL'][0]
        title = item["title"][0]
        condition = item['condition'][0]['conditionDisplayName'][0]
        price = item['sellingStatus'][0]["convertedCurrentPrice"][0]['__value__']
        if itemid in itemids:
            print 'item already alerted'
        else:
            with open('itemid.txt', 'a') as itemfile:
                itemfile.write(itemid + '\n')
            sendemail(url, title, price)


item already alerted
item already alerted
item already alerted
item already alerted
item already alerted
item already alerted
item already alerted
